In [1]:
%load_ext autoreload
%autoreload 2
%load_ext tensorboard
%matplotlib inline

In [2]:
from utils import *
from scale_layer import *
from datetime import date

In [3]:
trainloader, testloader, classes = load_CIFAR()

Files already downloaded and verified
Files already downloaded and verified


### Batchnorm Training
- `model_bn` is the model with batchnorm layers
- train this model on the CIFAR 10 dataset

In [4]:
loss_history_bn = []
writer_bn = SummaryWriter(log_dir='runs/bn_stats-test-%s' % date.today(), flush_secs=10)

In [5]:
model_bn = nn.Sequential(
    nn.Conv2d(3, 16, 3),
    ScaleLayer(tanh=True, writer=writer_bn),
    nn.FractionalMaxPool2d(2, output_ratio=(1/np.sqrt(2))),
    nn.Conv2d(16, 32, 3),
    ScaleLayer(tanh=True, writer=writer_bn),
    nn.FractionalMaxPool2d(2, output_ratio=(1/np.sqrt(2))),
    nn.Flatten(),
    nn.Linear(5408, 1152),
    ScaleLayer(tanh=True, writer=writer_bn),
    nn.Linear(1152, 512),
    ScaleLayer(tanh=True, writer=writer_bn),
    nn.Linear(512, 128),
    ScaleLayer(tanh=True, writer=writer_bn),
    nn.Linear(128, 10)
)

In [6]:
ce_loss = nn.CrossEntropyLoss(reduction='sum')
def criterion(op, y, model_p):
    params = list(model_p.parameters())
    return ce_loss(op, y) #+ (1/len(params))*(torch.norm(torch.cat([param.view(-1) for param in params]))) #+ torch.sum((1/len(params))*torch.exp(torch.abs(torch.cat([param.view(-1) for param in params]))))


In [7]:
loss_tanh = train_model(model_bn, trainloader, criterion, optimizer = optim.Adam(list(model_bn.parameters()), lr=3e-4), epochs=20, writer=writer_bn)

[1,   300] loss: 83.895
[1,   600] loss: 81.017
[2,   300] loss: 83.816
[2,   600] loss: 67.662
[3,   300] loss: 64.859
[3,   600] loss: 71.525
[4,   300] loss: 76.311
[4,   600] loss: 58.988
[5,   300] loss: 61.730
[5,   600] loss: 50.550
[6,   300] loss: 53.798
[6,   600] loss: 56.084
[7,   300] loss: 74.826
[7,   600] loss: 48.872
[8,   300] loss: 41.761
[8,   600] loss: 55.253
[9,   300] loss: 45.908
[9,   600] loss: 34.489
[10,   300] loss: 39.807
[10,   600] loss: 59.676
[11,   300] loss: 50.701
[11,   600] loss: 38.134
[12,   300] loss: 29.604
[12,   600] loss: 45.058
[13,   300] loss: 42.479
[13,   600] loss: 37.061
[14,   300] loss: 24.695
[14,   600] loss: 32.002
[15,   300] loss: 30.886
[15,   600] loss: 29.199
[16,   300] loss: 28.073
[16,   600] loss: 29.647
[17,   300] loss: 31.322
[17,   600] loss: 15.605
[18,   300] loss: 22.783
[18,   600] loss: 20.256
[19,   300] loss: 27.406
[19,   600] loss: 29.945
[20,   300] loss: 25.233
[20,   600] loss: 22.347


In [10]:
stats_bn = print_stats(model_bn)

Mean:  0.039082933
Standard Deviation:  0.03185012
Sparsity:  0.0017200133864245003


In [11]:
bn_model_err = get_model_error(model_bn, testloader)

test error: 0.3285
